In [9]:
import pandas as pd

ECLO_data = pd.read_csv('train.csv')

In [10]:
ECLO_data.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


In [11]:
print(ECLO_data['ECLO'].value_counts())

ECLO
3     18675
5      6743
6      4967
1      2578
9      1528
8      1152
4       818
10      611
12      611
11      394
15      283
7       263
13      194
2       150
14      145
16      101
18       89
17       78
20       39
19       36
21       33
24       18
23       17
22       17
26       12
27       11
25        7
28        6
30        5
31        4
34        3
32        3
37        3
33        2
35        2
36        1
40        1
65        1
56        1
47        1
57        1
45        1
52        1
74        1
39        1
66        1
Name: count, dtype: int64


In [12]:
ECLO_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  object
 3   기상상태         39609 non-null  object
 4   시군구          39609 non-null  object
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

In [35]:
from sklearn.model_selection import train_test_split

# ID, 가해운전자 차종, 가해운전자 성별, 피해운전자 차종, 피해운전자 성별, 삭제
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    df_copy.drop(['ID','가해운전자 차종', '가해운전자 성별', '피해운전자 차종', '피해운전자 성별','사망자수','중상자수','경상자수','부상자수'], axis=1, inplace = True)
    return df_copy

In [36]:
def get_train_test_dataset(df=None):
    df_copy = get_preprocessed_df(df)
    X_features = df_copy.iloc[:, :-1]
    y_target=df_copy.iloc[:, -1]
    
    X_train, X_test, y_train, y_test =\
    train_test_split(X_features, y_target, test_size=0.3, random_state=0)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_dataset(ECLO_data)

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def get_clf_eval(y_test, pred, pred_proba):
    # 오차 행렬, 정확도, 정밀도, 재현율, F1 스코어, ROC-AUC를 출력하는 함수
# 오차 행렬을 계산합니다. 
# 오차 행렬은 실제 레이블과 예측 레이블을 비교하여 True Positives, False Positives,
# True Negatives, False Negatives의 수를 표시합니다.
    confusion = confusion_matrix(y_test, pred)

# 정확도를 계산합니다.
# 정확도는 전체 데이터 중에서 올바르게 예측된 데이터의 비율입니다.
    accuracy = accuracy_score(y_test, pred)

# 정밀도를 계산합니다.
# 정밀도는 양성으로 예측된 데이터 중 실제로 양성인 데이터의 비율입니다.
    precision = precision_score(y_test, pred)

# 재현율을 계산합니다.
# 재현율(민감도)는 실제 양성 데이터 중 모델이 양성으로 예측한 데이터의 비율입니다.
    recall = recall_score(y_test, pred)

# F1 스코어를 계산합니다.
# F1 스코어는 정밀도와 재현율의 조화 평균이며, 두 지표의 균형을 나타냅니다.
    f1 = f1_score(y_test, pred)

# ROC-AUC 스코어를 계산합니다.
# ROC-AUC는 모델의 수신자 조작 특성 곡선(ROC) 아래의 면적을 측정하며,
# 1에 가까울수록 좋은 성능을 나타냅니다.
    roc_auc = roc_auc_score(y_test, pred_proba)

# 계산된 성능 지표들을 출력합니다.
    print('오차 행렬')
    print(confusion)
    print('\n정확도: {:.4f}, 정밀도: {:.4f}, 재현율: {:.4f}, F1: {:.4f}, ROC-AUC: {:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  int64 
 3   기상상태         39609 non-null  int32 
 4   시군구          39609 non-null  int32 
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

In [18]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
lr_pred_proba = lr_clf.predict_proba(X_test)[:,1]

get_clf_eval(y_test, lr_pred, lr_pred_proba)

ValueError: could not convert string to float: '2019-02-02 14'

In [19]:
print(ECLO_data['요일'].value_counts())

요일
금요일    6179
화요일    6023
수요일    5969
월요일    5895
목요일    5759
토요일    5670
일요일    4114
Name: count, dtype: int64


In [20]:
ECLO_data['요일'].replace('월요일',0,inplace=True)
ECLO_data['요일'].replace('화요일',1,inplace=True)
ECLO_data['요일'].replace('수요일',2,inplace=True)
ECLO_data['요일'].replace('목요일',3,inplace=True)
ECLO_data['요일'].replace('금요일',4,inplace=True)
ECLO_data['요일'].replace('토요일',5,inplace=True)
ECLO_data['요일'].replace('일요일',6,inplace=True)

In [23]:
print(ECLO_data['요일'].value_counts())

요일
4    6179
1    6023
2    5969
0    5895
3    5759
5    5670
6    4114
Name: count, dtype: int64


In [25]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [24]:
print(ECLO_data['기상상태'].value_counts())

기상상태
맑음    36181
비      2627
흐림      729
기타       56
안개        8
눈         8
Name: count, dtype: int64


In [28]:
le.fit_transform(ECLO_data['기상상태'])
le.fit_transform(ECLO_data['기상상태'].unique())

array([2, 5, 0, 3, 4, 1])

In [30]:
ECLO_data['기상상태'] = le.fit_transform(ECLO_data['기상상태'])

In [31]:
print(ECLO_data['기상상태'].value_counts())

기상상태
2    36181
3     2627
5      729
0       56
4        8
1        8
Name: count, dtype: int64


In [32]:
ECLO_data['시군구'].value_counts()

시군구
대구광역시 남구 대명동      1749
대구광역시 수성구 범어동     1173
대구광역시 달서구 상인동     1143
대구광역시 서구 비산동       968
대구광역시 서구 내당동       938
                  ... 
대구광역시 중구 도원동         2
대구광역시 동구 신용동         2
대구광역시 중구 동성로1가       2
대구광역시 북구 도남동         2
대구광역시 동구 평광동         1
Name: count, Length: 199, dtype: int64

In [33]:
ECLO_data['시군구'] = le.fit_transform(ECLO_data['시군구'])

In [34]:
ECLO_data['시군구'].value_counts()

시군구
0      1749
128    1173
11     1143
112     968
111     938
       ... 
158       2
66        2
161       2
93        2
78        1
Name: count, Length: 199, dtype: int64

In [ ]:
ECLO_data['시군구'] = le.fit_transform(ECLO_data['시군구'])